# Using OpenAI models with Strands Agent

## Overview

Strands Agents is an SDK that takes a model-driven approach to building and running AI agents in just a few lines of code. Strands supports multiple providers and models hosted anywhere.

[LiteLLM](https://docs.litellm.ai/docs/) is a unified interface for various LLM providers that allows you to interact with models from Amazon, Anthropic, OpenAI, and many others through a single API. The Strands Agent SDK implements a LiteLLM provider, allowing you to run agents against any model LiteLLM supports.

In this example, we will show you how to use `gpt-4.1-mini` model hosted in Microsoft Azure as the underlying model in your Strands Agent. We will use a simple agent use case with a weather and a get time tool.


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |LiteLLM model                                      |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **LiteLLM model**: using a model provided via LiteLLM

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* Azure Account
* gpt-4.1-mini access

Let's now install the requirement packages for our Strands Agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 154.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 144.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [litellm]m3/4 [litellm]


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel

### Setting up Azure keys

Let's now setup the Azure API Keys

In [3]:
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

### Setting up custom tools

Let's now setup two dummy tools to test our agent

In [4]:
@tool
def current_time(timezone: str = "UTC") -> str:
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    # Dummy implementation. Please replace with actual weather API call.
    return "sunny"

### Defining agent underlying LLM model

Next let's define our agent underlying model using LiteLLM. We will set it to `gpt-4.1-mini`

In [5]:
model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)

### Defining Agent

Now that we have all the required information available, let's define our agent

In [6]:
system_prompt = "You are a simple agent that can tell the time and the weather"
agent = Agent(
    model=litellm_model,
    system_prompt=system_prompt,
    tools=[current_time, current_weather],
)

### Testing agent

Let's now invoke the agent to test it

In [7]:
results = agent("What time is it in Seattle? And how is the weather?")


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



APIConnectionError: litellm.APIConnectionError: AzureException APIConnectionError - list index out of range
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/litellm/main.py", line 1194, in completion
    optional_params = get_optional_params(
                      ^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/litellm/utils.py", line 3766, in get_optional_params
    optional_params = litellm.AzureOpenAIConfig().map_openai_params(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/litellm/llms/azure/chat/gpt_transformation.py", line 165, in map_openai_params
    api_version_month = api_version_times[1]
                        ~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


#### Analysing the agent's results

Nice! We've invoked our agent for the first time! Let's now explore the results object. First thing we can see is the messages being exchanged by the agent in the agent's object

In [ ]:
agent.messages

Next we can take a look at the usage of our agent for the last query by analysing the result `metrics`

In [ ]:
results.metrics

### Congratulations!

In this notebook you learned how to use LiteLLM with OpeanAi serving answers for weather agent.